Try to estimate the key of a song using the pitch space model


In [1]:
import circularPitchSpace as cps
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import librosa.display
import librosa
import madmom
import IPython


In [1]:
path = ["/home/max/ET-TI/Masterarbeit/datasets/guitarset/audio_mono-pickup_mix/00_SS3-98-C_comp_mix.wav",
        "/home/max/ET-TI/Masterarbeit/datasets/beatles/audio/12_-_Let_It_Be/03_-_Across_the_Universe.mp3",
        "/home/max/ET-TI/Masterarbeit/prototyping/data/moonlight_sonata_minor.mp3",
]
path = path[1]
time = 30
dcp =  madmom.audio.chroma.DeepChromaProcessor()
chroma = dcp(path)
chroma = chroma[:time*10,:]
fig,ax = plt.subplots(1,3,figsize=(9,3))
cps.plotCircleOfFifths(ax[0])
r_F,r_FR,r_TR,r_DR = cps.transformChroma(chroma)
r_max = np.max(np.abs(r_F))
r_F_norm = (r_F/r_max)*1j  # scale and rotate by 90° for plot!
for col in range(r_F_norm.shape[0]):
    ax[0].plot(r_F_norm[col].real,r_F_norm[col].imag,'ob',markersize=.3)

pitch_class_energy = cps.getPitchClassEnergyProfile(chroma,0.8,0.8)
# find chroma bin with maximum pitch class energy
key_maxima = np.argmax(pitch_class_energy,axis=1)
# count number of maximas for each pitch_class index
key_probabilities = np.bincount(key_maxima,minlength=12) / chroma.shape[0]
key_est = cps.pitch_classes[np.argmax(key_probabilities)]
print(f"Estimated Pitch Class: {key_est.name}")
# Major or Minor? 
# investigate with angle of FR -> left Major, right Minor
r_FR_est = r_FR[:,key_est.pitch_class_index]
cps.plotCircleOfFifthsRelated(ax[1],key_est.pitch_class_index)
r_max = np.max(np.abs(r_FR_est))
for col in range(r_FR_est.shape[0]):
    z = r_FR_est[col]/r_max*1j
    ax[1].plot(z.real,z.imag,'ob',markersize=.3)
cps.fillCircle(ax[1],np.pi/2,3*np.pi/2,'green')
cps.fillCircle(ax[1],np.pi/2,-np.pi/2,'red')

r_TR_est = r_TR[:,key_est.pitch_class_index]
cps.plotCircleOfThirdsRelated(ax[2],key_est.pitch_class_index)
r_max = np.max(np.abs(r_TR_est))
for col in range(r_TR_est.shape[0]):
    z = r_TR_est[col]/r_max*1j
    ax[2].plot(z.real,z.imag,'ob',markersize=.3)
 
angle = np.angle(r_FR[:,key_est.pitch_class_index])
print(f"major chords: #{np.count_nonzero(angle>0)}")
print(f"minor: #{np.count_nonzero(angle<=0)}")
if np.count_nonzero(angle>0) >= np.count_nonzero(angle<=0):
    quality = 'maj'
    print(f"Estimated Key: {key_est.name} major")
else:
    quality = "min"
    print(f"Estimated Key: {key_est.name} minor")



NameError: name 'madmom' is not defined

Tonsetenergie </br>
$E(k) = \frac{\sum_{n_c=-6}^5(\epsilon_{n_c}*z)^2}{\sum_{n_c=-6}^5(\epsilon_{n_c})^2}$ </br>
z = 1| nc ist die Tonigkeit des Tonsets k </br>
z = 0| nc ist nicht Tonigkeit des Tonsets k  </br>
Die Tonsetenergie wird mit einem Prozentsatz der Gesamtenergie verglichen, und nur Tonsets mit hoher Energie werden für die weitere Analyse betrachtet. Wird beispielsweise ein C-Dur akkord gespielt, so ist die Tonsetenergie in den Tonarten C,F und G größer als in allen anderen Tonsets. </br>
Winkelgewicht: Der C-Dur akkord ist leitereigen in mehreren Tonarten, als Tonika in C,Subdominante in G und Dominante in F. Mit einer Winkelgewichtung bei der Analyse kann diese Mehrdeutigkeit umgangen werden, da die Akkordfunktion anhand des Winkel des Summenvektors im Tonartbezogenen Terzkreis TR bestimmt werden</br>

